**Notes**: [Notion link](https://tallmanofkaramana.notion.site/Build-a-Large-Language-Model-From-Scratch-Sebastian-Raschka-18d345517e4b8014a489d6880f91640f)

In [1]:
import torch
torch.__version__

'2.4.0'

## Input sequence
- A tensor of tokens after converting them into vector embeddings of dimenstion `3`.

In [2]:
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],  # Your     (x^1)
        [0.55, 0.87, 0.66],  # journey  (x^2)
        [0.57, 0.85, 0.64],  # starts   (x^3)
        [0.22, 0.58, 0.33],  # with     (x^4)
        [0.77, 0.25, 0.10],  # one      (x^5)
        [0.05, 0.80, 0.55],  # step     (x^6)
    ]
)

## Computing a single attention (for one token)

In [3]:
attention_scores_2 = torch.empty(inputs.shape[0]) # initializing to the length of no. of tokens
q = inputs[1] # query

for i, x_i in enumerate(inputs):
    attention_scores_2[i] = torch.dot(x_i, q) # The dot product of all input tokens with the query will give the importance of each token relative to the query.

print(attention_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- Internally, a dot product would multiply each element of the embedding for a given input token with the corresponding element from the embedding of the query.
- Same is illustrated below. It quantifies how closely two vectors are aligned: a higher dot product indicates a greater degree of alignment or similarity between the vectors

In [4]:
result = 0

for idx, elem in enumerate(inputs[1]): # iterates through the embedding of the second token only.
    result += elem * q[idx]

print(result) # Manually worked out dot product.
print(torch.dot(inputs[1], q)) # Dot product

tensor(1.4950)
tensor(1.4950)


- Attention scores are normalized (so that they sum up to 1) to obtain attention weights.

In [5]:
attention_wts_2 = attention_scores_2 / attention_scores_2.sum() # manually normalizing the scores.
attention_wts_2

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [6]:
# Defining a naive softmax implementation.
def naive_softmax(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [7]:
attention_wts_2 = naive_softmax(attention_scores_2)
attention_wts_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [8]:
# Using the softmax from PyTorch.
attention_wts_2 = torch.softmax(attention_scores_2, dim=0)
attention_wts_2

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

- Computing the context vector for the second token.

In [9]:
context_vec_2 = torch.zeros(q.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attention_wts_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


## Computing Attention for all input tokens

In [10]:
input_size = inputs.shape[0]
attention_scores = torch.empty(inputs.shape[0], inputs.shape[0])

In [11]:
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attention_scores[i, j] = torch.dot(x_i, x_j)

attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

- The same result is obtained by matrix multiplication (`for` loops tend to be slow).

In [12]:
attention_scores = inputs @ inputs.T
attention_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [13]:
attention_wts = torch.softmax(attention_scores, dim=-1) # normalized along column axis, hence dim=-1. i.e., sum across columns for each row.
attention_wts

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [14]:
print(f"{inputs.shape=}, {attention_wts.shape=}")

inputs.shape=torch.Size([6, 3]), attention_wts.shape=torch.Size([6, 6])


In [15]:
context_vectors = attention_wts @ inputs # 6x6 by 6x3.
context_vectors

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [16]:
context_vectors.shape

torch.Size([6, 3])

## Implementing self-attention with trainable weights

In [17]:
x_2 = inputs[1]
dim_in = inputs.shape[1]
dim_out = 2 # for pedantic reasons.

# Typically, for GPT `dim_in` is same as `dim_out`. The dimenstions in this example are deliberately set to different values to follow the computation. 

In [18]:
torch.manual_seed(123)

- Initializing the weight matrices for query, key and value.

In [19]:
W_q = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_k = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_v = torch.nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)

In [20]:
W_q.shape, x_2.shape

(torch.Size([3, 2]), torch.Size([3]))

In [21]:
q_2 = x_2 @ W_q
k_2 = x_2 @ W_k
v_2 = x_2 @ W_v

In [22]:
k_2

tensor([0.4433, 1.1419])

In [23]:
q_2

tensor([0.4306, 1.4551])

In [24]:
v_2

tensor([0.3951, 1.0037])

- Computing the keys and values for all the tokens from the input sequence.

In [25]:
keys = inputs @ W_k
values = inputs @ W_v

In [26]:
print(f"{keys.shape=}, {values.shape=}")

keys.shape=torch.Size([6, 2]), values.shape=torch.Size([6, 2])


- Computing the attention scores.

In [27]:
attention_score_2 = q_2 @ keys.T
attention_score_2.shape

torch.Size([6])

In [28]:
attention_score_2

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])

In [29]:
# attention_wt_2 = torch.softmax(attention_score_2, dim=-1)
# attention_wt_2

- Alternate normalization is using scaled dot product.
- This is called *scaled dot product attention*. The rationale behind this is as the dot product increases, the behaviour is similar to a step function (saturated) and therefore results in smaller gradients and thereby slowing down the training process.

In [30]:
attention_wt_2 = torch.softmax(attention_score_2 / (keys.shape[-1] ** 0.5), dim=-1)
attention_wt_2

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])

In [31]:
context_vec_2 = attention_wt_2 @ values
context_vec_2

tensor([0.3061, 0.8210])

## Generic implementation of Self Attention

In [32]:
from SelfAttention import SelfAttention_v1, SelfAttention_v2
print(dim_in, dim_out)

3 2


In [33]:
torch.manual_seed(123)

sa_v1 = SelfAttention_v1(dim_in, dim_out, scale=True)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [34]:
torch.manual_seed(789)

sa_v2 = SelfAttention_v2(dim_in, dim_out, scale=True)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


### EXERCISE 3.1
- Correctly assign the weights from an instance of `SelfAttention_v2` to an instance of `SelfAttention_v1`.

In [35]:
sa_v2.W_q.weight, sa_v2.W_q.weight.shape

(Parameter containing:
 tensor([[ 0.3161,  0.4568,  0.5118],
         [-0.1683, -0.3379, -0.0918]], requires_grad=True),
 torch.Size([2, 3]))

In [36]:
sa_v1.W_q, sa_v1.W_q.shape

(Parameter containing:
 tensor([[0.2961, 0.5166],
         [0.2517, 0.6886],
         [0.0740, 0.8665]], requires_grad=True),
 torch.Size([3, 2]))

Dimension of `sa_v1` is `3x2`, while `sa_v2` is `2x3`. Therefore, `sa_v2` weights may be transposed before assigning them to `sa_v1`.

In [37]:
sa_v2.W_q.weight.T, sa_v2.W_q.weight.T.shape

(tensor([[ 0.3161, -0.1683],
         [ 0.4568, -0.3379],
         [ 0.5118, -0.0918]], grad_fn=<PermuteBackward0>),
 torch.Size([3, 2]))

In [38]:
sa_v1_mod = SelfAttention_v1(dim_in, dim_out, scale=True)
sa_v1_mod(inputs)

tensor([[1.0177, 0.5637],
        [1.0427, 0.5773],
        [1.0428, 0.5774],
        [1.0153, 0.5620],
        [1.0286, 0.5701],
        [1.0162, 0.5622]], grad_fn=<MmBackward0>)

In [39]:
sa_v1_mod.W_q.shape == sa_v2.W_q.weight.T.shape

True

In [40]:
sa_v1_mod.W_q.shape, sa_v2.W_q.weight.T.shape

(torch.Size([3, 2]), torch.Size([3, 2]))

In [41]:
## Wrong way to copy the weights.

# sa_v1_mod.W_q.values = sa_v2.W_q.weight.T.values
# sa_v1_mod.W_k.values = sa_v2.W_k.weight.T.values
# sa_v1_mod.W_v.values = sa_v2.W_v.weight.T.values

## Correct way to copy the weights. Use torch.nn.Parameter.
sa_v1_mod.W_q = torch.nn.Parameter(sa_v2.W_q.weight.T)
sa_v1_mod.W_k = torch.nn.Parameter(sa_v2.W_k.weight.T)
sa_v1_mod.W_v = torch.nn.Parameter(sa_v2.W_v.weight.T)

In [42]:
sa_v1_mod(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

In [43]:
sa_v2(inputs)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)

## Causal Attention

In [44]:
batch_input = torch.stack([inputs, inputs], dim=0)
batch_input.shape

torch.Size([2, 6, 3])

In [46]:
torch.manual_seed(123)

from SelfAttention import MaskedAttention

context_length = batch_input.shape[1]
masked_attention = MaskedAttention(dim_in, dim_out, context_length, dropout=0.0, scale=True)
masked_ctx_vec = masked_attention(batch_input)

In [47]:
masked_ctx_vec.shape

torch.Size([2, 6, 2])